In [1]:
## HYPERPARAMETERS
BATCH_SIZE = 4
EPOCHS = 30
LEARNING_RATE = 0.001
HIDDEN_SIZE = 16

In [2]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
## Data Preparation
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

In [4]:
class TextDataset(Dataset):
    def __init__(self, data_path):
        self.df = pd.read_csv(data_path)
        self.vectorizer = CountVectorizer()
        self.bow_features = self.vectorizer.fit_transform(self.df["text"]).toarray()
        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(self.df["intent"])
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        bow_features = torch.tensor(self.bow_features[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return bow_features, label
    
    def get_labels(self):
        return self.label_encoder.classes_
    
    def get_vocab_size(self):
        vocab_size = len(self.vectorizer.vocabulary_)
        return vocab_size
    
    def get_num_classes(self):
        return len(self.label_encoder.classes_)
    
    def label2id(self, label):
        return self.label_encoder.classes_[label]

In [5]:
import yaml

def load_config(config_path: str) -> dict:
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
    return config

load_config("../config/config.yaml")

{'data': {'train_path': 'data/train',
  'val_path': 'data/val',
  'batch_size': 8,
  'num_workers': 1},
 'model': {'hidden_size': 8, 'dropout': 0.2},
 'training': {'epochs': 10,
  'learning_rate': 0.001,
  'device': 'cpu',
  'save_dir': 'checkpoints'},
 'logging': {'log_dir': 'logs', 'experiment_name': 'ann_classification'}}

In [6]:
## Train code
def train_epoch(model, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        logits = model(data)
        
        loss = criterion(logits, target)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
        total_loss += loss.item()
        
        pred_label = logits.softmax(-1).argmax(-1) # [batch_size label]
        
        _, predicted = logits.max(1) # [same as above]
        # print("*******************************")
        # print(logits)
        # print("======================================")
        # print(logits.max(1))
        # print("***********************************")
        # print(pred_label)
        # print("-------------------------------------------")
        total += target.size(0)
        correct += predicted.eq(target).sum().item()
        
    return total_loss / total, correct / total


In [7]:
def train(model, train_loader, optimizer, criterion):
    for epoch in range(EPOCHS):
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion)
        print(f"Epoch {epoch} | Loss {train_loss} | Accuracy {train_acc}")

In [8]:
## Defining Model

class TextClassifier(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.layer_1 = torch.nn.Linear(in_features=input_size, out_features=hidden_size)
        self.layer_2 = torch.nn.Linear(in_features=hidden_size, out_features=hidden_size)
        self.layer_3 = torch.nn.Linear(in_features=hidden_size, out_features=num_classes)
        self.relu = torch.nn.ReLU()
        
    def forward(self, X):
        X = self.relu(self.layer_1(X))
        X = self.relu(self.layer_2(X))
        X = self.layer_3(X)
        
        return X

In [9]:
train_dataset = TextDataset("../data/train.csv")
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [10]:
model = TextClassifier(
    # bow feature size will be the input size here
    input_size=train_dataset.get_vocab_size(),
    hidden_size=HIDDEN_SIZE,
    num_classes=train_dataset.get_num_classes()
)

In [11]:
model

TextClassifier(
  (layer_1): Linear(in_features=70, out_features=16, bias=True)
  (layer_2): Linear(in_features=16, out_features=16, bias=True)
  (layer_3): Linear(in_features=16, out_features=3, bias=True)
  (relu): ReLU()
)

In [12]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [13]:
train(model, train_loader, optimizer, criterion)

Epoch 0 | Loss 0.317097544670105 | Accuracy 0.35294117647058826
Epoch 1 | Loss 0.3157257227336659 | Accuracy 0.35294117647058826
Epoch 2 | Loss 0.3148645232705509 | Accuracy 0.35294117647058826
Epoch 3 | Loss 0.31375794901567344 | Accuracy 0.35294117647058826
Epoch 4 | Loss 0.32749736309051514 | Accuracy 0.35294117647058826
Epoch 5 | Loss 0.32623359035043153 | Accuracy 0.35294117647058826
Epoch 6 | Loss 0.324672635863809 | Accuracy 0.35294117647058826
Epoch 7 | Loss 0.30921011461931114 | Accuracy 0.35294117647058826
Epoch 8 | Loss 0.31990691844154806 | Accuracy 0.35294117647058826
Epoch 9 | Loss 0.3073955563937916 | Accuracy 0.35294117647058826
Epoch 10 | Loss 0.316697779823752 | Accuracy 0.35294117647058826
Epoch 11 | Loss 0.3056673863354851 | Accuracy 0.35294117647058826
Epoch 12 | Loss 0.3153080484446357 | Accuracy 0.35294117647058826
Epoch 13 | Loss 0.31132567279479084 | Accuracy 0.35294117647058826
Epoch 14 | Loss 0.300165081725401 | Accuracy 0.35294117647058826
Epoch 15 | Loss 0.